In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('/Users/alokroy/Documents/Programming/Projects/Amex/data/data/train_data_new.csv')
test_data = pd.read_csv('/Users/alokroy/Documents/Programming/Projects/Amex/data/data/test_data_new.csv')

In [3]:
train_data.head()

,team_wicket_loss_average,team_count_50runs_last15,team_winp_last5,team_bowler_performance_ratio,team_strike_rate,team_avg_wicket,team_batting_strength,team_win_rate_at_venue,team_batsman_performance_index,team_overall_win_rate,...,team_avg_eco,team_avg_score_ground_ratio,avg_score_ground_batting_first_ratio,match_experience_score_ratio,boundary_frequency_ratio,avg_runs_powerplay_ratio,total_wickets_bowlers_recent_ratio,total_runs_top_batsmen_recent_ratio,total_runs_top_batsmen_ratio,winner_01
0,-1.368785,1.210788,0.672131,1.244528,2.148438,1.238807,1.173089,7.071068e-01,1.972308,-0.083475,...,-0.554419,-0.104744,0.046338,1.368785,0.738186,0.115396,1.316632,1.178372,1.478076,1
1,-1.041201,0.564618,1.952381,-0.461919,-1.112081,-0.521669,-0.091446,1.490116e-08,-1.107182,0.854515,...,-0.285641,0.191746,0.416732,0.975199,-0.595941,0.247339,-0.893293,-0.088788,1.107182,0
2,0.485610,-0.232356,0.672131,0.615018,-0.497557,0.352594,-0.035741,1.490116e-08,-0.369545,-0.667032,...,-0.172086,0.349779,-0.418176,-1.494046,-0.244610,-0.470772,0.618219,-0.035741,-1.396318,0
3,0.003970,1.432296,1.975610,-0.485610,1.162619,-0.527744,1.285775,1.490116e-08,0.762741,0.324563,...,-0.531796,0.274726,1.279749,-0.453102,0.375219,0.913194,-0.406649,1.291848,-0.220136,0
4,1.589567,-0.294019,1.327869,0.366713,0.222848,0.041039,-0.080819,1.000000e+00,0.225562,-0.738186,...,0.204372,-0.776981,-0.846914,-0.435573,-0.476694,0.521669,0.068874,0.206596,-0.461919,1


In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

def plot_density(train_data, test_data, column_name):

    title = f"'Density Plot of {column_name}"
    xlabel = column_name
    ylabel = 'Density'
   
    plt.figure(figsize=(10, 6))
    sns.kdeplot(train_data[column_name], fill=True, label='Train Data')
    sns.kdeplot(test_data[column_name], fill=True, label='Test Data')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [5]:
# plot_density(train_data, test_data, 'team_wicket_loss_average')

In [6]:
# for col in train_data.columns:
#     plot_density(train_data, test_data, col)

In [7]:
train_data.corr()['winner_01'].sort_values()

avg_score_ground_batting_first_ratio   -0.226530
team_avg_score_ground_ratio            -0.219863
team_win_percentage_fielding_first     -0.219447
match_experience_score_ratio           -0.219131
team_batting_strength                  -0.190013
total_runs_top_batsmen_recent_ratio    -0.186817
team_batsman_runs_recent               -0.186817
total_runs_top_batsmen_ratio           -0.182399
team_bowler_performance_ratio          -0.166911
team_batsman_performance_recent        -0.160832
team_avg_wicket                        -0.157641
team_pom_frequency                     -0.157227
team_avg_wickets_top5_bowlers          -0.151704
team_count_50runs_last15               -0.147812
total_wickets_bowlers_recent_ratio     -0.145209
boundary_frequency_ratio               -0.143416
avg_runs_powerplay_ratio               -0.142774
team_strike_rate                       -0.132729
team_count_100runs_ratio_last15        -0.126637
team_batsman_performance_index         -0.110905
team_SR             

## Feature Selection

### 1. Recursive Feature Elimination (RFE)

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.impute import SimpleImputer

# Define the features and target variable
X = train_data.drop('winner_01', axis=1)  # Assuming 'winner_01' is the target variable
y = train_data['winner_01']

# Handle missing values by imputing with the median
imputer = SimpleImputer(strategy='median')
X_imputed = imputer.fit_transform(X)

# Convert the imputed array back to a DataFrame
X_imputed_df = pd.DataFrame(X_imputed, columns=X.columns)

# Split the dataset into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X_imputed_df, y, test_size=0.0, random_state=42)
X_train = X_imputed_df
y_train = y

# Create a RandomForest classifier to use with RFE
model = RandomForestClassifier(n_estimators=100, random_state=42)

# RFE with RandomForest for the top 15 features
rfe = RFE(estimator=model, n_features_to_select=10)
rfe.fit(X_train, y_train)

# Get the selected features
selected_features = X.columns[rfe.support_]

# Print selected features
print("Selected features:", selected_features)

# Model training can be done here using the selected features
# For example:
# X_train_selected = X_train[selected_features]
# X_val_selected = X_val[selected_features]


Selected features: Index(['team_bowler_performance_ratio', 'team_batting_strength',
       'team_pom_frequency', 'team_win_percentage_fielding_first',
       'team_avg_runs_1st_inning', 'team_SR', 'team_avg_eco',
       'team_avg_score_ground_ratio', 'avg_score_ground_batting_first_ratio',
       'match_experience_score_ratio'],
      dtype='object')


Selected features: Index(['team_bowler_performance_ratio', 'team_strike_rate', 'team_avg_wicket',
       'team_batting_strength', 'team_batsman_performance_recent',
       'team_pom_frequency', 'team_win_percentage_fielding_first',
       'team_avg_runs_1st_inning', 'team_SR', 'team_avg_eco',
       'team_avg_score_ground_ratio', 'avg_score_ground_batting_first_ratio',
       'match_experience_score_ratio', 'boundary_frequency_ratio',
       'avg_runs_powerplay_ratio'],
      dtype='object')

In [9]:
# selected_features = train_data.corr()['winner_01'].sort_values()[:10].index

In [10]:
# selected_features

## Model

In [11]:
train_data.fillna(0, inplace=True)

In [12]:
df = train_data[selected_features]

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = df
# y = df['winner_01']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((758, 10), (758,), (190, 10), (190,))

In [15]:
# from catboost import CatBoostClassifier
# from mlxtend.feature_selection import SequentialFeatureSelector
# from sklearn.model_selection import cross_val_score
# import numpy as np

# # Initialize the CatBoost classifier
# catboost_model = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=5, verbose=0)

# # Variables to store the best results
# best_score = 0
# best_features = None
# best_k = 0

# # Convert X_train to a NumPy array if it is a DataFrame
# X_train_np = X_train.to_numpy() if hasattr(X_train, 'to_numpy') else X_train

# # Iterate over possible numbers of features
# for k in range(1, X_train.shape[1] + 1):
#     # Create the backward feature selector with k features
#     selector = SequentialFeatureSelector(estimator=catboost_model,
#                                          k_features=k,
#                                          forward=False,
#                                          floating=False,
#                                          verbose=0,
#                                          scoring="accuracy",
#                                          cv=5)
    
#     # Fit the feature selector
#     selector.fit(X_train_np, y_train)
    
#     # Evaluate the model with selected features
#     selected_features = list(selector.k_feature_idx_)
#     score = cross_val_score(catboost_model, X_train_np[:, selected_features], y_train, cv=5, scoring='accuracy').mean()
    
#     # Update best score and features if current is better
#     if score > best_score:
#         best_score = score
#         best_features = selected_features
#         best_k = k

# print(f"Best number of features: {best_k}")
# print(f"Best feature indices: {best_features}")
# print(f"Best cross-validation accuracy: {best_score}")

# # Fit the model with the best number of features
# catboost_model.fit(X_train_np[:, best_features], y_train)

# pred = catboost_model.predict(X_test[:, best_features])
# accuracy_score(y_test, pred)

In [16]:
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, y_train, X_test, y_test are defined

# Initialize models
GBM_model = GradientBoostingClassifier()
LGBM_model = LGBMClassifier()
XGB_model = XGBClassifier()
CatBoost_model = CatBoostClassifier()

# Train the models
GBM_model.fit(X_train, y_train)
LGBM_model.fit(X_train, y_train)
XGB_model.fit(X_train, y_train)
CatBoost_model.fit(X_train, y_train)

# Make predictions on test set
y_pred_GBM_test = GBM_model.predict(X_test)
y_pred_LGBM_test = LGBM_model.predict(X_test)
y_pred_XGB_test = XGB_model.predict(X_test)
y_pred_CatBoost_test = CatBoost_model.predict(X_test)

# Make predictions on train set
y_pred_GBM_train = GBM_model.predict(X_train)
y_pred_LGBM_train = LGBM_model.predict(X_train)
y_pred_XGB_train = XGB_model.predict(X_train)
y_pred_CatBoost_train = CatBoost_model.predict(X_train)

# Evaluate models on test data
accuracy_GBM_test = accuracy_score(y_test, y_pred_GBM_test)
accuracy_LGBM_test = accuracy_score(y_test, y_pred_LGBM_test)
accuracy_XGB_test = accuracy_score(y_test, y_pred_XGB_test)
accuracy_CatBoost_test = accuracy_score(y_test, y_pred_CatBoost_test)

# Evaluate models on train data
accuracy_GBM_train = accuracy_score(y_train, y_pred_GBM_train)
accuracy_LGBM_train = accuracy_score(y_train, y_pred_LGBM_train)
accuracy_XGB_train = accuracy_score(y_train, y_pred_XGB_train)
accuracy_CatBoost_train = accuracy_score(y_train, y_pred_CatBoost_train)

# Print accuracies
print("Accuracy for GBM model on training data:", accuracy_GBM_train)
print("Accuracy for GBM model on testing data:", accuracy_GBM_test)
print('====================================================================')
print("Accuracy for LGBM model on training data:", accuracy_LGBM_train)
print("Accuracy for LGBM model on testing data:", accuracy_LGBM_test)
print('====================================================================')
print("Accuracy for XGB model on training data:", accuracy_XGB_train)
print("Accuracy for XGB model on testing data:", accuracy_XGB_test)
print('====================================================================')
print("Accuracy for CatBoost model on training data:", accuracy_CatBoost_train)
print("Accuracy for CatBoost model on testing data:", accuracy_CatBoost_test)

[LightGBM] [Info] Number of positive: 386, number of negative: 372
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2526
[LightGBM] [Info] Number of data points in the train set: 758, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.509235 -> initscore=0.036944
[LightGBM] [Info] Start training from score 0.036944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [17]:
import numpy as np
import pandas as pd
from itertools import combinations
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression

X_val = X_test
y_val = y_test

# Initialize the base models
xgb_model = XGBClassifier(n_estimators=50, learning_rate=0.05, random_state=42)
cat_model = CatBoostClassifier(iterations=50, learning_rate=0.05, depth=3, verbose=False, random_state=42)

# Create the bagging ensemble model
xgb_bagging = BaggingClassifier(estimator=xgb_model, n_estimators=10, random_state=42)
cat_bagging = BaggingClassifier(estimator=cat_model, n_estimators=10, random_state=42)

# Fit the models
xgb_bagging.fit(X_train, y_train)
cat_bagging.fit(X_train, y_train)

# Make predictions
xgb_bag_pred = xgb_bagging.predict_proba(X_val)[:, 1]
cat_bag_pred = cat_bagging.predict_proba(X_val)[:, 1]

# Weighted Averaging
# Assume XGB is weighted 0.6 and CatBoost 0.4 based on prior validation scores
weighted_pred = xgb_bag_pred * 0.4 + cat_bag_pred * 0.6
final_predictions_weighted = np.where(weighted_pred > 0.5, 1, 0)

# Stacking
estimators = [
    ('xgb', xgb_bagging),
    ('cat', cat_bagging)
]
stack_model = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stack_model.fit(X_train, y_train)
stack_pred = stack_model.predict(X_val)

# Evaluate the models
accuracy_weighted = accuracy_score(y_val, final_predictions_weighted)
accuracy_stacked = accuracy_score(y_val, stack_pred)

print("Accuracy of the weighted model:", accuracy_weighted)
print("Accuracy of the stacking model:", accuracy_stacked)

Accuracy of the weighted model: 0.6684210526315789
Accuracy of the stacking model: 0.6526315789473685


In [18]:
xgb_model = XGBClassifier(n_estimators=50, random_state=42)
cat_model = CatBoostClassifier(iterations=50, verbose=False, random_state=42)
gbm_model = GradientBoostingClassifier(n_estimators=50, random_state=42)
lgb_model = LGBMClassifier(n_estimators=50, random_state=42)

models = {
    'XGB': xgb_model,
    'CatBoost': cat_model,
    'GBM': gbm_model,
    'LightGBM': lgb_model
}

# Function to train and predict
def train_predict(models, X_train, y_train, X_val):
    predictions = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)[:, 1]
        predictions[name] = preds
    return predictions

# Evaluate combinations of models
def evaluate_combinations(X_train, y_train, X_val, y_val):
    model_predictions = train_predict(models, X_train, y_train, X_val)
    results = {}
    
    for r in range(1, len(models)+1):
        for combo in combinations(models.keys(), r):
            # Average predictions from models in the current combination
            combo_preds = np.mean([model_predictions[model] for model in combo], axis=0)
            final_predictions = np.where(combo_preds > 0.5, 1, 0)
            accuracy = accuracy_score(y_val, final_predictions)
            results[combo] = accuracy
    
    return results

# Main function to run the analysis
def main(X_train, X_test, y_train, y_test):
    X_train, X_val, y_train, y_val = X_train, X_test, y_train, y_test
    results = evaluate_combinations(X_train, y_train, X_val, y_val)
    
    for combo, accuracy in results.items():
        print(f"Models: {', '.join(combo)} - Accuracy: {accuracy:.4f}")

# Run the main function
main(X_train, X_test, y_train, y_test)

[LightGBM] [Info] Number of positive: 386, number of negative: 372
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000211 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2526
[LightGBM] [Info] Number of data points in the train set: 758, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.509235 -> initscore=0.036944
[LightGBM] [Info] Start training from score 0.036944
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best 

In [82]:
import optuna
from catboost import Pool
train_pool = Pool(data=X_train, label=y_train)
valid_pool = Pool(data=X_test, label=y_test)

# Define the objective function for hyperparameter tuning
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 200),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'depth': trial.suggest_int('depth', 3, 7),
        'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1.0),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 1.0),
        'random_strength': trial.suggest_loguniform('random_strength', 1e-3, 10.0),
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait': trial.suggest_int('od_wait', 10, 50),
        'verbose': 0  # Suppress output for tuning
    }
    
    model = CatBoostClassifier(**params)
    model.fit(train_pool)
    preds = model.predict(valid_pool)
    accuracy = accuracy_score(y_test, preds)
    
    return accuracy  # Minimize the negative accuracy

# Create and run the Optuna study
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)

# Get the best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

# Train the final model with the best parameters
best_model_catboost = CatBoostClassifier(
    iterations=best_params['iterations'],
    learning_rate=best_params['learning_rate'],
    depth=best_params['depth'],
    l2_leaf_reg=best_params['l2_leaf_reg'],
    border_count=best_params['border_count'],
    bagging_temperature=best_params['bagging_temperature'],
    random_strength=best_params['random_strength'],
    od_type=best_params['od_type'],
    od_wait=best_params['od_wait'],
    verbose=100  # To monitor the training process
)

best_model_catboost.fit(X_train, y_train)

# Evaluate the final model on the validation set
final_preds = best_model_catboost.predict(X_test)
final_accuracy = accuracy_score(y_test, final_preds)
print(final_accuracy)

[I 2024-06-13 22:41:41,751] A new study created in memory with name: no-name-a1d18b9f-32be-4c8c-8e08-16ff967940a4
/var/folders/z7/t3qmw89j637f4y58ls6cb_vh0000gn/T/ipykernel_39188/3053236595.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
/var/folders/z7/t3qmw89j637f4y58ls6cb_vh0000gn/T/ipykernel_39188/3053236595.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1.0),
/var/folders/z7/t3qmw89j637f4y58ls6cb_vh0000gn/T/ipykernel_39188/3053236595.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will b

Best parameters: {'iterations': 134, 'learning_rate': 0.00018252773305712697, 'depth': 6, 'l2_leaf_reg': 0.0012314643153008098, 'border_count': 108, 'bagging_temperature': 0.046217755894046024, 'random_strength': 0.1058457108867618, 'od_type': 'Iter', 'od_wait': 40}
0:	learn: 0.6931185	total: 722us	remaining: 96.1ms
100:	learn: 0.6900816	total: 118ms	remaining: 38.5ms
133:	learn: 0.6891073	total: 137ms	remaining: 0us
0.7157894736842105


In [83]:
# import optuna
# from catboost import CatBoostClassifier, Pool
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import accuracy_score

# # Define the objective function for hyperparameter tuning
# def objective(trial):
#     params = {
#         'iterations': trial.suggest_int('iterations', 100, 500),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
#         'depth': trial.suggest_int('depth', 1, 5),
#         'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 1.0),
#         'border_count': trial.suggest_int('border_count', 32, 255),
#         'bagging_temperature': trial.suggest_loguniform('bagging_temperature', 0.01, 1.0),
#         'random_strength': trial.suggest_loguniform('random_strength', 1e-3, 10.0),
#         'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
#         'od_wait': trial.suggest_int('od_wait', 10, 50),
#         'verbose': 0  # Suppress output for tuning
#     }
    
#     model = CatBoostClassifier(**params)
#     cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
#     return cv_scores.mean()  # Maximize the cross-validation accuracy

# # Create and run the Optuna study
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# # Get the best parameters
# best_params = study.best_params
# print(f"Best parameters: {best_params}")

# # Train the final model with the best parameters
# best_model_catboost = CatBoostClassifier(
#     iterations=best_params['iterations'],
#     learning_rate=best_params['learning_rate'],
#     depth=best_params['depth'],
#     l2_leaf_reg=best_params['l2_leaf_reg'],
#     border_count=best_params['border_count'],
#     bagging_temperature=best_params['bagging_temperature'],
#     random_strength=best_params['random_strength'],
#     od_type=best_params['od_type'],
#     od_wait=best_params['od_wait'],
#     verbose=100  # To monitor the training process
# )

# best_model_catboost.fit(X_train, y_train)

# # Evaluate the final model on the validation set
# final_preds = best_model_catboost.predict(X_test)
# final_accuracy = accuracy_score(y_test, final_preds)
# print(f"Final accuracy on the test set: {final_accuracy}")

In [84]:
pred_train = best_model_catboost.predict(X_train)
accuracy_score(y_train, pred_train)

0.7506596306068601

## XGB

In [ ]:
# import optuna
# import xgboost as xgb
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split, cross_val_score

# def objective(trial):
#     param = {
#         'verbosity': 0,
#         'objective': 'binary:logistic',
#         'booster': trial.suggest_categorical('booster', ['gbtree', 'dart']),
#         'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
#         'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0)
#     }

#     # Specific parameters for gbtree and dart
#     if param['booster'] in ['gbtree', 'dart']:
#         param['max_depth'] = trial.suggest_int('max_depth', 3, 9)
#         param['eta'] = trial.suggest_loguniform('eta', 1e-8, 1.0)
#         param['gamma'] = trial.suggest_loguniform('gamma', 1e-8, 1.0)
#         param['grow_policy'] = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])

#     # Additional dart-specific parameters
#     if param['booster'] == 'dart':
#         param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
#         param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
#         param['rate_drop'] = trial.suggest_loguniform('rate_drop', 1e-8, 1.0)
#         param['skip_drop'] = trial.suggest_loguniform('skip_drop', 1e-8, 1.0)

#     # The gblinear booster doesn't have specific hyperparameters that are typically tuned

#     clf = xgb.XGBClassifier(**param)

#     score = cross_val_score(clf, X_train, y_train, n_jobs=-1, cv=3, scoring='accuracy').mean()
#     return score

# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)

# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

# # Best parameters
# best_params = study.best_trial.params
# print(f"Best parameters: {best_params}")

# # Retrain the model with the best parameters on the full training data
# best_model_XGB = xgb.XGBClassifier(**best_params)
# best_model_XGB.fit(X_train, y_train)

# # Predict the labels of the test set
# y_pred = best_model_XGB.predict(X_test)

# # Calculate the accuracy on the test set
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Accuracy on test set: {accuracy:.2f}")

In [ ]:
# import optuna
# import xgboost as xgb

# # Define objective function
# def objective(trial):
#     param = {
#         'objective': 'binary:logistic',
#         'eval_metric': 'logloss',
#         'booster': 'gbtree',
#         'verbosity': 0,
#         'n_estimators': trial.suggest_int('n_estimators', 100, 800),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-3),
#         'max_depth': trial.suggest_int('max_depth', 1, 4),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
#         'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
#         'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
#         'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
#         'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
#     }
    
#     model = xgb.XGBClassifier(**param)
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     return accuracy

# # Run optimization
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=1000)

# # Get the best parameters
# best_params = study.best_params
# print("Best Params:", best_params)

# # Train the final model with best parameters
# best_model_XGB = xgb.XGBClassifier(**best_params)
# best_model_XGB.fit(X_train, y_train)

# # Evaluate on the test set
# y_pred = best_model_XGB.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Test Accuracy:", accuracy)

In [ ]:
import optuna
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Define objective function
def objective(trial):
    param = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'booster': 'gbtree',
        'verbosity': 0,
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-3),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
    }
    
    model = xgb.XGBClassifier(**param)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    return cv_scores.mean()  # Maximize the cross-validation accuracy

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best parameters
best_params = study.best_params
print("Best Params:", best_params)

# Train the final model with best parameters
best_model_XGB = xgb.XGBClassifier(**best_params)
best_model_XGB.fit(X_train, y_train)

# Evaluate on the test set
y_pred = best_model_XGB.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

In [ ]:
pred_train = best_model_XGB.predict(X_train)
accuracy_score(y_train, pred_train)

## GBM

In [ ]:
# def objective(trial):
#     # Define the hyperparameters to be tuned
#     n_estimators = trial.suggest_int('n_estimators', 50, 500)
#     learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
#     max_depth = trial.suggest_int('max_depth', 3, 10)
    
#     # Create a GBM classifier with the suggested parameters
#     classifier = GradientBoostingClassifier(
#         n_estimators=n_estimators,
#         learning_rate=learning_rate,
#         max_depth=max_depth,
#         random_state=42
#     )
    
#     # Train the model
#     classifier.fit(X_train, y_train)
    
#     # Predict on the validation set
#     y_pred = classifier.predict(X_test)
    
#     # Calculate the accuracy
#     accuracy = accuracy_score(y_test, y_pred)
    
#     return accuracy

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=50)

# # Print the best parameters
# print("Best trial:")
# trial = study.best_trial

# print(f" Accuracy: {trial.value:.3f}")
# print(" Params: ")
# for key, value in trial.params.items():
#     print(f"    {key}: {value}")

# # Train the final model with the best parameters
# best_params = study.best_params
# best_model_GBM = GradientBoostingClassifier(**best_params)
# best_model_GBM.fit(X_train, y_train)

# # Evaluate the final model on the test set
# final_preds = best_model_GBM.predict(X_test)
# final_accuracy = accuracy_score(y_test, final_preds)
# print(f"Final accuracy: {final_accuracy:.3f}")

In [ ]:
# import optuna
# from sklearn.ensemble import GradientBoostingClassifier
# def objective(trial):
#     param = {
#         'n_estimators': trial.suggest_int('n_estimators', 100, 800),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
#         'max_depth': trial.suggest_int('max_depth', 1, 5),
#         'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
#         'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
#         'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
#         'max_features': trial.suggest_uniform('max_features', 0.5, 1.0)
#     }
    
#     model = GradientBoostingClassifier(**param)
#     model.fit(X_train, y_train)
    
#     # Predict on the validation set (or test set) and calculate accuracy
#     y_pred = model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
    
#     return accuracy

# # Run optimization
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=200)

# # Get the best parameters
# best_params = study.best_params
# print("Best Params:", best_params)

# # Train the final model with best parameters
# best_model_GBM = GradientBoostingClassifier(**best_params)
# best_model_GBM.fit(X_train, y_train)

# # Evaluate on the test set
# y_pred = best_model_GBM.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print("Test Accuracy:", accuracy)

In [ ]:
import optuna
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Define objective function
def objective(trial):
    param = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'max_depth': trial.suggest_int('max_depth', 1, 5),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'max_features': trial.suggest_uniform('max_features', 0.5, 1.0)
    }
    
    model = GradientBoostingClassifier(**param)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    
    return cv_scores.mean()  # Maximize the cross-validation accuracy

# Run optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Get the best parameters
best_params = study.best_params
print("Best Params:", best_params)

# Train the final model with best parameters
best_model_GBM = GradientBoostingClassifier(**best_params)
best_model_GBM.fit(X_train, y_train)

# Evaluate on the test set
y_pred = best_model_GBM.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

In [ ]:
pred_train = best_model_GBM.predict(X_train)
accuracy_score(y_train, pred_train)

## LGBM

In [ ]:
# import lightgbm as lgb
# from sklearn.metrics import log_loss

# def objective(trial):
#     # Specify the search space of hyperparameters
#     param = {
#         'objective': 'binary',
#         'metric': 'binary_logloss',
#         'verbosity': -1,
#         'boosting_type': 'gbdt',
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 256),
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'learning_rate': trial.suggest_loguniform('learning_rate', 0.005, 0.2)
#     }

#     # Train the model
#     gbm = lgb.train(param, lgb.Dataset(X_train, label=y_train), valid_sets=[lgb.Dataset(X_test, label=y_test)], num_boost_round=100)

#     # Predict on the test set
#     preds = gbm.predict(X_test)
#     # Compute log loss
#     loss = log_loss(y_test, preds)
#     return loss

# # Create a study object and optimize the objective function
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=100)

# # Print the best parameters
# print('Best trial:', study.best_trial.params)

# # Train the final model with the best parameters
# best_params = study.best_params
# best_model_LGBM = lgb.LGBMClassifier(**best_params)
# best_model_LGBM.fit(X_train, y_train)

# # Evaluate the final model on the test set
# final_preds = best_model_LGBM.predict(X_test)
# final_accuracy = accuracy_score(y_test, final_preds)
# print(f"Final accuracy: {final_accuracy:.3f}")

In [ ]:
# from lightgbm import LGBMClassifier
# def objective(trial):
#     param = {
#     'objective': 'binary',
#     'metric': 'binary_logloss',
#     'boosting_type': 'gbdt',
#     'verbosity': -1,
#     'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
#     'num_leaves': trial.suggest_int('num_leaves', 20, 50),
#     'max_depth': trial.suggest_int('max_depth', 1, 3),
#     'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 200),
#     'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#     'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#     'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
#     'n_estimators': trial.suggest_int('n_estimators', 100, 1000)  # You can adjust the range as needed
# }


#     model = LGBMClassifier(**param)
#     model.fit(X_train, y_train)

#     preds = model.predict(X_test)
#     accuracy = accuracy_score(y_test, preds)

#     return accuracy

# # Create Optuna study and optimize
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=200)

# # Print best trial
# print(f'Number of finished trials: {len(study.trials)}')
# print(f'Best trial: {study.best_trial.params}')

# # Train the final model with the best parameters
# best_params = study.best_trial.params
# best_params['objective'] = 'binary'
# best_params['metric'] = 'binary_logloss'
# best_params['verbosity'] = -1

# best_model_LGBM = LGBMClassifier(**best_params)
# best_model_LGBM.fit(X_train, y_train)
# y_pred= best_model_LGBM.predict(X_test)
# accuracy = accuracy_score(y_test, best_model_LGBM.predict(X_test))

# print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
import optuna
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Define the objective function
def objective(trial):
    param = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 50),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 100, 200),
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000)  # You can adjust the range as needed
    }

    model = LGBMClassifier(**param)
    # Perform cross-validation and return the mean accuracy
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy')
    return cv_scores.mean()

# Create Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

# Print best trial
print(f'Number of finished trials: {len(study.trials)}')
print(f'Best trial: {study.best_trial.params}')

# Train the final model with the best parameters
best_params = study.best_trial.params
best_params['objective'] = 'binary'
best_params['metric'] = 'binary_logloss'
best_params['verbosity'] = -1

best_model_LGBM = LGBMClassifier(**best_params)
best_model_LGBM.fit(X_train, y_train)

# Evaluate on the test set
y_pred = best_model_LGBM.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

In [ ]:
pred_train = best_model_LGBM.predict(X_train)
accuracy_score(y_train, pred_train)

## Test

In [49]:
best_pred = np.array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
                    0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
                    0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1,
                    0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
                    0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
                    0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
                    0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
                    1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,
                    1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1,
                    0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
                    1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
                    0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
                    0, 0, 1, 1, 0, 0, 0])

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt

def analyze_model(final_model, X, y):
    # Preprocess the data
    df_test = test_data[selected_features]
    df_test.fillna(0, inplace=True)
    
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X_test_scaled = scaler.transform(X_test)
    test_scaled = scaler.transform(df_test)
    
    # Train the model
    final_model.fit(X, y)
    
    # Predict on training set
    pred_X = final_model.predict(X)
    train_accuracy = accuracy_score(y, pred_X)
   
    
    # Predict on validation set
    pred_train = final_model.predict(X_train)
    train_accuracy = accuracy_score(y_train, pred_train)
     
    
    # Predict on test set
    pred_test = final_model.predict(X_test_scaled)
    test_accuracy = accuracy_score(y_test, pred_test)
    
    # Predict on new test data
    preds = final_model.predict(test_scaled)
    
    # Calculate probability predictions
    pred_proba = final_model.predict_proba(test_scaled)
    
    # Plot probability distribution
    sns.histplot(pred_proba, kde=True)
    plt.title('Prediction Probability Distribution')
    plt.show()
    
    # Function to count zeros in predictions
    def count_zeros(predictions):
        total_elements = len(predictions)
        count_zeros = np.sum(predictions == 0)
        percentage_zeros = (count_zeros / total_elements) * 100
        return percentage_zeros
    
    print(f"Predictions: {preds}")
    
    print(f"Accuracy on X set: {train_accuracy:.2f}")
    print(f"Accuracy on X_train set: {train_accuracy:.2f}")
    print(f"Accuracy on X_test set: {test_accuracy:.2f}")
    print(f"Percentage of zeros in the test set: {count_zeros(preds):.2f}%")
    print(f"Percentage of zeros in the best prediction: {count_zeros(best_pred):.2f}%")
    
    # Compare predictions with best_pred
    comparison_result = np.where(best_pred != preds, -1, 0)
    mismatched_indexes = np.where(comparison_result == -1)[0]
    
    print(f"Number of Mismatched indexes: {len(mismatched_indexes)}")

In [ ]:
# analyze_model(final_model=best_model_catboost, X=X, y=y)

In [ ]:
# analyze_model(final_model=best_model_XGB, X=X, y=y)

In [ ]:
# analyze_model(final_model=best_model_GBM, X=X, y=y)

In [ ]:
# analyze_model(final_model=best_model_LGBM, X=X, y=y)

In [ ]:
# analyze_model(final_model=best_model_XGB, X=X, y=y)

## Ensemble 

In [ ]:
models = {
    'XGB': best_model_XGB,
    'CatBoost': best_model_catboost,
    'GBM': best_model_GBM,
    'LightGBM': best_model_LGBM
}

# Function to train and predict
def train_predict(models, X_train, y_train, X_val):
    predictions = {}
    for name, model in models.items():
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)[:, 1]
        predictions[name] = preds
    return predictions

def get_test_predictions(models, X, y, test, model_set):
    selected_models = {name: models[name] for name in model_set if name in models}
    model_predictions = train_predict(selected_models, X, y, test)
    
    # Average predictions from the specified models
    combined_preds = np.mean(list(model_predictions.values()), axis=0)
    final_predictions = np.where(combined_preds > 0.5, 1, 0)
    
    return final_predictions, combined_preds

# Evaluate combinations of models
def evaluate_combinations(X_train, y_train, X_val, y_val):
    model_predictions = train_predict(models, X_train, y_train, X_val)
    results = {}
    
    for r in range(1, len(models)+1):
        for combo in combinations(models.keys(), r):
            # Average predictions from models in the current combination
            combo_preds = np.mean([model_predictions[model] for model in combo], axis=0)
            final_predictions = np.where(combo_preds > 0.5, 1, 0)
            accuracy = accuracy_score(y_val, final_predictions)
            results[combo] = accuracy
    
    return results

# Main function to run the analysis
def main(X_train, X_test, y_train, y_test):
    X_train, X_val, y_train, y_val = X_train, X_test, y_train, y_test
    results = evaluate_combinations(X_train, y_train, X_val, y_val)
    
    for combo, accuracy in results.items():
        print(f"Models: {', '.join(combo)} - Accuracy: {accuracy:.4f}")

# Run the main function
main(X_train, X_test, y_train, y_test)

In [90]:
df_test = test_data[selected_features]
df_test.fillna(0, inplace=True)

scaler = StandardScaler()
X= scaler.fit_transform(X)
test = scaler.transform(df_test)

/var/folders/z7/t3qmw89j637f4y58ls6cb_vh0000gn/T/ipykernel_39188/1287964554.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test.fillna(0, inplace=True)
/opt/homebrew/Caskroom/miniforge/base/envs/amex/lib/python3.12/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [91]:
best_model_catboost.fit(X, y)
# best_model_XGB.fit(X, y)
# best_model_GBM.fit(X, y)
# best_model_LGBM.fit(X, y)

0:	learn: 0.6931181	total: 835us	remaining: 111ms
100:	learn: 0.6900814	total: 78.4ms	remaining: 25.6ms
133:	learn: 0.6890940	total: 101ms	remaining: 0us


In [92]:
catboost_preds = best_model_catboost.predict_proba(test)[:, 1]
# gbm_preds = best_model_GBM.predict_proba(test)[:, 1]
# xgb_preds = best_model_XGB.predict_proba(test)[:, 1]
# lgbm_preds = best_model_LGBM.predict_proba(test)[:, 1]

# Averaging the predictions
average_preds = (catboost_preds) /1

# Binarize the averaged probabilities
final_preds_prob = np.where(average_preds > 0.5, 1, 0)

In [93]:
(final_preds_prob == 0).sum()/271

0.5313653136531366

In [94]:
(final_preds_prob != best_pred).sum()

30

In [95]:
final_preds_prob

array([1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0])

In [96]:
from scipy.stats import mode

catboost_preds = best_model_catboost.predict(test)
gbm_preds = best_model_GBM.predict(test)
xgb_preds = best_model_XGB.predict(test)
lgbm_preds = best_model_LGBM.predict(test)

# Collect predictions in an array
preds = np.array([catboost_preds, xgb_preds,gbm_preds, lgbm_preds])

# Apply majority voting
final_preds, _ = mode(preds, axis=0)
final_preds_vote = final_preds.ravel()

NameError: name 'best_model_GBM' is not defined

In [ ]:
final_preds_vote

In [ ]:
(final_preds_vote == 0).sum()/271

In [ ]:
(final_preds_vote != best_pred).sum()

In [ ]:
(final_preds_prob != final_preds_vote).sum()

## Sub

In [ ]:
df_test = pd.read_csv('/Users/alokroy/Documents/Programming/Projects/Amex/data/main/6644a1e287df6_test_data_with_samplefeatures.csv')
df_test['winner'] = final_preds_prob
# df_test['win_pred_score'] = np.max(pred_proba, axis=1)

winner_id = []
for i in range(len(df_test)):
  if df_test['winner'][i] == 0:
    winner_id.append(df_test['team1_id'][i])
  else:
    winner_id.append(df_test['team2_id'][i])

df_test['win_pred_team_id'] = winner_id

In [ ]:
df_test.to_csv('check.csv', index=False)